### Arabic Egyptian Text Classifier 
Based on Dataset of Tweets from twitter classified into positive and negative texts 

In [128]:
import numpy as np  
import pandas as pd  
import os
# for dirname, _, filenames in os.walk('C:/Users/Abdullah/Desktop/archive'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
### load data 
neg_train=pd.read_csv('train_Arabic_tweets_negative_20190413.tsv',sep='\t')
neg_train.columns=['sentiment','text']
len(neg_train)
pos_train=pd.read_csv('train_Arabic_tweets_positive_20190413.tsv',sep='\t')
pos_train.columns=['sentiment','text']
print(len(pos_train))
print(len(neg_train))

22760
22513


In [31]:
pos_train.head()

,sentiment,text
0,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...
1,pos,من الخير نفسه 💛
2,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
3,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...
4,pos,#الاتحاد_النصر لاتحسبونا نسينا يالطواقي ولانبي...


In [32]:
neg_train.head()

,sentiment,text
0,neg,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
1,neg,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...
2,neg,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...
3,neg,الدودو جايه تكمل علي 💔
4,neg,أحببته حتى أقنعنى إن ما فات من العمر كان إنتظا...


In [3]:
## concatenate data 
data=pd.concat([neg_train,pos_train])
len(data)

45273

In [4]:
## shufle data
from sklearn.utils import shuffle
shuffled_data = shuffle(data)
shuffled_data.head()

,sentiment,text
8305,neg,اي معجب سري بعد 🤔
22198,pos,اعلق احلامي على حافة #الغيم ان طحنا #غيث وان ب...
6516,pos,خربت عليك 😜 👇😂😂
10638,pos,يسعد صبااحك 🌹
6470,pos,الي كاتبو رب العباد راح تشوفو العين والأجدر يف...


In [5]:
## convert pos to 1 and neg to 0
shuffled_data['sentiment']=shuffled_data['sentiment'].apply(lambda x: 1 if x =='pos' else 0)

In [6]:
shuffled_data.head()

,sentiment,text
8305,0,اي معجب سري بعد 🤔
22198,1,اعلق احلامي على حافة #الغيم ان طحنا #غيث وان ب...
6516,1,خربت عليك 😜 👇😂😂
10638,1,يسعد صبااحك 🌹
6470,1,الي كاتبو رب العباد راح تشوفو العين والأجدر يف...


In [7]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from keras.models import Model ,Sequential
from keras.layers import Embedding ,Dense ,Dropout , LSTM ,Input ,Activation , Bidirectional ,GlobalMaxPool1D
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import glorot_uniform
%matplotlib inline

Using TensorFlow backend.
c:\users\abdullah\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\abdullah\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\abdullah\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [8]:
from gensim.models import KeyedVectors
def load_w2v(file_path,binary):
    return KeyedVectors.load_word2vec_format(file_path,binary=binary)

In [9]:
w2v = load_w2v("wiki.arz.vec", binary=False)

In [10]:
print(len(w2v.vocab))

54595


In [11]:
sample=w2v['حسن']
print(sample.shape)
print(w2v.most_similar('حسن'))

(300,)
[('محسن', 0.7458980083465576), ('حسن،', 0.7444434762001038), ('بحسن', 0.743319034576416), ('وحسن', 0.7386170029640198), ('لحسن', 0.7364068031311035), ('حسنك', 0.7337351441383362), ('حسنه', 0.7143272161483765), ('حسناء', 0.7094866633415222), ('يحسن', 0.6863618493080139), ('حسني', 0.6850252747535706)]


In [12]:
import os 
import glob
import codecs
from keras.preprocessing.text import Tokenizer ,text_to_word_sequence
def get_stop_words():
    path='list.txt'
    stop_words = []
    with codecs.open(path, "r", encoding="utf-8", errors="ignore") as myfile:
        stop_words = myfile.readlines()
    stop_words = [word.strip() for word in stop_words]
    return stop_words
stop_words=get_stop_words()
print(stop_words)

['،', 'ء', 'ءَ', 'آ', 'آب', 'آذار', 'آض', 'آل', 'آمينَ', 'آناء', 'آنفا', 'آه', 'آهاً', 'آهٍ', 'آهِ', 'أ', 'أبدا', 'أبريل', 'أبو', 'أبٌ', 'أجل', 'أجمع', 'أحد', 'أخبر', 'أخذ', 'أخو', 'أخٌ', 'أربع', 'أربعاء', 'أربعة', 'أربعمئة', 'أربعمائة', 'أرى', 'أسكن', 'أصبح', 'أصلا', 'أضحى', 'أطعم', 'أعطى', 'أعلم', 'أغسطس', 'أفريل', 'أفعل به', 'أفٍّ', 'أقبل', 'أكتوبر', 'أل', 'ألا', 'ألف', 'ألفى', 'أم', 'أما', 'أمام', 'أمامك', 'أمامكَ', 'أمد', 'أمس', 'أمسى', 'أمّا', 'أن', 'أنا', 'أنبأ', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'أنتِ', 'أنشأ', 'أنه', 'أنًّ', 'أنّى', 'أهلا', 'أو', 'أوت', 'أوشك', 'أول', 'أولئك', 'أولاء', 'أولالك', 'أوّهْ', 'أى', 'أي', 'أيا', 'أيار', 'أيضا', 'أيلول', 'أين', 'أيّ', 'أيّان', 'أُفٍّ', 'ؤ', 'إحدى', 'إذ', 'إذا', 'إذاً', 'إذما', 'إذن', 'إزاء', 'إلى', 'إلي', 'إليكم', 'إليكما', 'إليكنّ', 'إليكَ', 'إلَيْكَ', 'إلّا', 'إمّا', 'إن', 'إنَّ', 'إى', 'إياك', 'إياكم', 'إياكما', 'إياكن', 'إيانا', 'إياه', 'إياها', 'إياهم', 'إياهما', 'إياهن', 'إياي', 'إيهٍ', 'ئ', 'ا', 'ا?', 'ا?ى', 'االا', 'االتى', 'اب

In [13]:
## remove stop words from text
shuffled_data['text']=shuffled_data['text'].apply(lambda x : [item for item in x.split() if item not in stop_words])

In [14]:
shuffled_data.text

8305                                        [معجب, سري, 🤔]
22198    [اعلق, احلامي, حافة, #الغيم, طحنا, #غيث, بقينا...
6516                                        [خربت, 😜, 👇😂😂]
10638                                    [يسعد, صبااحك, 🌹]
6470     [كاتبو, رب, العباد, تشوفو, العين, والأجدر, يفو...
                               ...                        
12487                  [شفته, بعقلي, وعيني, إلا, تدمعه, 💙]
21794       [حسن, عيوبك, يصفعك, احداهم, مدعيا, المعرفة, 😶]
2684     [لي, صاحب, اغلا, النفس, بشوي, ويزيد, باقي, الم...
763                         [هذاك, الحين, الوضع, حفالي, 💔]
16772    [يقلك, قصيره, لبست, كعب, زكمت, لييه؟؟, تغير, ا...
Name: text, Length: 45273, dtype: object

In [44]:
shuffled_data['text'].head()

8305                                        [معجب, سري, 🤔]
22198    [اعلق, احلامي, حافة, #الغيم, طحنا, #غيث, بقينا...
6516                                        [خربت, 😜, 👇😂😂]
10638                                    [يسعد, صبااحك, 🌹]
6470     [كاتبو, رب, العباد, تشوفو, العين, والأجدر, يفو...
Name: text, dtype: object

In [15]:
max_sequence_length=85
max_nm_words=len(w2v.vocab)
embeding_dim=300

####  fit_on_texts
Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency.
#### texts_to_sequences
Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index

In [16]:
## tokenize data
tokenizer= Tokenizer()
tokenizer.fit_on_texts(shuffled_data['text'])
word_index=tokenizer.word_index
vocab_size=len(word_index)
print(vocab_size)

83181


In [17]:
train_sequences=tokenizer.texts_to_sequences(shuffled_data['text'])
train_paded_sequences=pad_sequences(train_sequences,maxlen=max_sequence_length,padding='post',truncating='post')

In [18]:
len(train_paded_sequences)

45273

In [19]:
from sklearn.model_selection import train_test_split
train_paded_sequences,valid_paded_sequences,y_train,y_valid=train_test_split(train_paded_sequences,shuffled_data['sentiment'].values,test_size=.2)

In [20]:
### build model
sentence_indices = Input(shape=(max_sequence_length,),dtype='int32')
embedding_layer = Embedding(vocab_size+1 , embeding_dim,  input_length=max_sequence_length)
embeddings = embedding_layer(sentence_indices)   
X = LSTM(60, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(embeddings)
X = GlobalMaxPool1D()(X)
X = Dropout(0.2)(X)
X = Dense(128)(X)
X = Activation("relu")(X)
X = Dropout(0.2)(X)
X = Dense(512)(X)
X = Activation("relu")(X)
X = Dropout(0.2)(X)
X = Dense(1)(X)
X = Activation('softmax')(X)
model = Model(inputs=sentence_indices,outputs=X)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=['accuracy'])

In [22]:
history = model.fit(train_paded_sequences, y_train, batch_size=32, epochs=2, validation_data=(valid_paded_sequences, y_valid))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 36218 samples, validate on 9055 samples
Epoch 1/2
36218/36218 [==============================] - 750s 21ms/step - loss: 7.9175 - acc: 0.5034 - val_loss: 7.9686 - val_acc: 0.5002
Epoch 2/2
36218/36218 [==============================] - 826s 23ms/step - loss: 7.9175 - acc: 0.5034 - val_loss: 7.9686 - val_acc: 0.5002


In [23]:
model.save('model.h5')

## Load The model and predict 

In [24]:
from keras.models import load_model
model = load_model('model.h5')